# Mod 4 Project - Starter Notebook

This notebook has been provided to you so that you can make use of the following starter code to help with the trickier parts of preprocessing the Zillow dataset. 

The notebook contains a rough outline the general order you'll likely want to take in this project. You'll notice that most of the areas are left blank. This is so that it's more obvious exactly when you should make use of the starter code provided for preprocessing. 

**_NOTE:_** The number of empty cells are not meant to infer how much or how little code should be involved in any given step--we've just provided a few for your convenience. Add, delete, and change things around in this notebook as needed!

# Some Notes Before Starting

This project will be one of the more challenging projects you complete in this program. This is because working with Time Series data is a bit different than working with regular datasets. In order to make this a bit less frustrating and help you understand what you need to do (and when you need to do it), we'll quickly review the dataset formats that you'll encounter in this project. 

## Wide Format vs Long Format

If you take a look at the format of the data in `zillow_data.csv`, you'll notice that the actual Time Series values are stored as separate columns. Here's a sample: 

<img src='https://raw.githubusercontent.com/learn-co-students/dsc-mod-4-project-seattle-ds-102819/master/images/df_head.png'>

You'll notice that the first seven columns look like any other dataset you're used to working with. However, column 8 refers to the median housing sales values for April 1996, column 9 for May 1996, and so on. This This is called **_Wide Format_**, and it makes the dataframe intuitive and easy to read. However, there are problems with this format when it comes to actually learning from the data, because the data only makes sense if you know the name of the column that the data can be found it. Since column names are metadata, our algorithms will miss out on what dates each value is for. This means that before we pass this data to our ARIMA model, we'll need to reshape our dataset to **_Long Format_**. Reshaped into long format, the dataframe above would now look like:

<img src='https://raw.githubusercontent.com/learn-co-students/dsc-mod-4-project-seattle-ds-102819/master/images/melted1.png'>

There are now many more rows in this dataset--one for each unique time and zipcode combination in the data! Once our dataset is in this format, we'll be able to train an ARIMA model on it. The method used to convert from Wide to Long is `pd.melt()`, and it is common to refer to our dataset as 'melted' after the transition to denote that it is in long format. 

# Helper Functions Provided

Melting a dataset can be tricky if you've never done it before, so you'll see that we have provided a sample function, `melt_data()`, to help you with this step below. Also provided is:

* `get_datetimes()`, a function to deal with converting the column values for datetimes as a pandas series of datetime objects
* Some good parameters for matplotlib to help make your visualizations more readable. 

Good luck!


# Step 1: Load the Data/Filtering for Chosen Zipcodes

In [1]:
import pandas as pd

__Comments__\
Region name is the zipcode.
RegionID not relevant to the analysis.

In [2]:
df = pd.read_csv('Zillow_data/zhvi_all-homes_zipcode.csv')
df.head(3)

,RegionID,SizeRank,RegionName,RegionType,StateName,State,City,Metro,CountyName,1996-01-31,...,2019-09-30,2019-10-31,2019-11-30,2019-12-31,2020-01-31,2020-02-29,2020-03-31,2020-04-30,2020-05-31,2020-06-30
0,61639,0,10025,Zip,NY,NY,New York,New York-Newark-Jersey City,New York County,233265.0,...,1248340.0,1234262.0,1229890.0,1226466.0,1208024.0,1182758.0,1150900.0,1134880.0,1120949.0,1112549.0
1,84654,1,60657,Zip,IL,IL,Chicago,Chicago-Naperville-Elgin,Cook County,211748.0,...,494425.0,493485.0,492514.0,491726.0,491562.0,492618.0,494017.0,494766.0,494546.0,494435.0
2,61637,2,10023,Zip,NY,NY,New York,New York-Newark-Jersey City,New York County,245773.0,...,1161916.0,1153259.0,1156287.0,1175142.0,1193746.0,1205413.0,1203165.0,1209735.0,1211403.0,1212520.0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30443 entries, 0 to 30442
Columns: 303 entries, RegionID to 2020-06-30
dtypes: float64(294), int64(3), object(6)
memory usage: 70.4+ MB


In [4]:
percent_zip_codes = str(round((30443/41702)*100,2))
print ('The data set includes ' + percent_zip_codes + '% of US zip codes')

The data set includes 73.0% of US zip codes


__Notes__
- Dropping Metro column because of missing values
- State and StateName are dupliates
- RegionID, RegionType not relevant

In [5]:
#rename RegionName to ZipCode
df.rename(columns={'RegionName': 'zipcode'}, inplace=True)
#drop RegionID
df.drop(columns=['RegionID', 'RegionType', 'StateName', 'SizeRank'], inplace=True)
df.head(2)

,zipcode,State,City,Metro,CountyName,1996-01-31,1996-02-29,1996-03-31,1996-04-30,1996-05-31,...,2019-09-30,2019-10-31,2019-11-30,2019-12-31,2020-01-31,2020-02-29,2020-03-31,2020-04-30,2020-05-31,2020-06-30
0,10025,NY,New York,New York-Newark-Jersey City,New York County,233265.0,231813.0,231891.0,231866.0,232396.0,...,1248340.0,1234262.0,1229890.0,1226466.0,1208024.0,1182758.0,1150900.0,1134880.0,1120949.0,1112549.0
1,60657,IL,Chicago,Chicago-Naperville-Elgin,Cook County,211748.0,211591.0,211180.0,211044.0,210477.0,...,494425.0,493485.0,492514.0,491726.0,491562.0,492618.0,494017.0,494766.0,494546.0,494435.0


In [6]:
def melt_data_monthly_mean(df):
    melted = pd.melt(df, id_vars=['zipcode', 'City', 'State', 'CountyName', 'Metro'], var_name='time')
    melted['time'] = pd.to_datetime(melted['time'], infer_datetime_format=True)
    melted = melted.dropna(subset=['value'])
    return melted.groupby('time').aggregate({'value':'mean'})

In [7]:
melt_data_monthly_mean(df) #this is the mean price by month

,value
time,
1996-01-31,124979.207863
1996-02-29,124606.091938
1996-03-31,124543.916218
1996-04-30,124573.163709
1996-05-31,124632.428195
...,...
2020-02-29,227757.147127
2020-03-31,228465.291561
2020-04-30,229086.276320


In [8]:
def melt_data(df):
    melted = pd.melt(df, id_vars=['zipcode', 'City', 'State', 'CountyName', 'Metro'], var_name='time')
    melted['time'] = pd.to_datetime(melted['time'], infer_datetime_format=True)
    melted = melted.dropna(subset=['value'])
    return melted

#value_name='df_melted'

In [9]:
df = melt_data(df)
df.head(2)

,zipcode,City,State,CountyName,Metro,time,value
0,10025,New York,NY,New York County,New York-Newark-Jersey City,1996-01-31,233265.0
1,60657,Chicago,IL,Cook County,Chicago-Naperville-Elgin,1996-01-31,211748.0


__Missing Values__
- the only column with missing values is the 'Metro' column, so it is reasonable to assume that these rows are not in a metro area. Missing values were replaced with 'rural'.

In [10]:
df.isna().sum()

zipcode             0
City                0
State               0
CountyName          0
Metro         1252330
time                0
value               0
dtype: int64

In [11]:
df.fillna('rural', inplace=True)

In [12]:
df.isna().sum()

zipcode       0
City          0
State         0
CountyName    0
Metro         0
time          0
value         0
dtype: int64

In [13]:
df.dtypes

zipcode                int64
City                  object
State                 object
CountyName            object
Metro                 object
time          datetime64[ns]
value                float64
dtype: object

In [14]:
df.dtypes.unique()

array([dtype('int64'), dtype('O'), dtype('<M8[ns]'), dtype('float64')],
      dtype=object)

In [15]:
df['value'] = df['value'].astype('int')

In [16]:
df.dtypes

zipcode                int64
City                  object
State                 object
CountyName            object
Metro                 object
time          datetime64[ns]
value                  int64
dtype: object

# Step 2: Data Preprocessing

In [17]:
def get_datetimes(df):
    return pd.to_datetime(df.columns.values[1:], format='%Y-%m')

In [18]:
# get_datetimes(df)

# Step 3: EDA and Visualization

__EDA Questions__
1. Which 3 zip codes had the most volatile prices?
2. Which 3 zip codes had the most stable prices?
3. Which zip codes are seeing price decay?
4. Which (if any) zip codes saw price growth during the 2008-2012 recession?
5. Which zip codes had stable prices during the 2008-2012 recession?
6. Which zip codes are priced the lowest?
7. Which zip codes are priced the highest?
8. How much higher, on average, are prices in metro areas?
9. What was the net change in overall median price from 2009-2019?
10. Which zip codes had a percent increase greater than inflation in the last year?

What do different regions look like?
What are the top cities?
Where was the top growth in the last 5 years, 10 years?

In [19]:
df.nlargest(3, ['value'])

,zipcode,City,State,CountyName,Metro,time,value
8840789,94027,Atherton,CA,San Mateo County,San Francisco-Oakland-Hayward,2020-03-31,6518578
8871232,94027,Atherton,CA,San Mateo County,San Francisco-Oakland-Hayward,2020-04-30,6515411
8810346,94027,Atherton,CA,San Mateo County,San Francisco-Oakland-Hayward,2020-02-29,6459116


In [20]:
font = {'family' : 'normal',
        'weight' : 'bold',
        'size'   : 22}

matplotlib.rc('font', **font)

# NOTE: if you visualizations are too cluttered to read, try calling 'plt.gcf().autofmt_xdate()'!

NameError: name 'matplotlib' is not defined

# Step 4: Reshape from Wide to Long Format

In [21]:
def melt_data(df):
    melted = pd.melt(df, id_vars=['RegionName', 'City', 'State', 'Metro', 'CountyName'], var_name='time')
    melted['time'] = pd.to_datetime(melted['time'], infer_datetime_format=True)
    melted = melted.dropna(subset=['value'])
    return melted.groupby('time').aggregate({'value':'mean'})

# Get Zipcodes with Highest Growth

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6844134 entries, 0 to 8950241
Data columns (total 7 columns):
 #   Column      Dtype         
---  ------      -----         
 0   zipcode     int64         
 1   City        object        
 2   State       object        
 3   CountyName  object        
 4   Metro       object        
 5   time        datetime64[ns]
 6   value       int64         
dtypes: datetime64[ns](1), int64(2), object(4)
memory usage: 417.7+ MB


In [23]:
df.head()

,zipcode,City,State,CountyName,Metro,time,value
0,10025,New York,NY,New York County,New York-Newark-Jersey City,1996-01-31,233265
1,60657,Chicago,IL,Cook County,Chicago-Naperville-Elgin,1996-01-31,211748
2,10023,New York,NY,New York County,New York-Newark-Jersey City,1996-01-31,245773
3,77494,Katy,TX,Harris County,Houston-The Woodlands-Sugar Land,1996-01-31,200430
4,60614,Chicago,IL,Cook County,Chicago-Naperville-Elgin,1996-01-31,286382


In [42]:
#Filter to only show the 2 months we want
df_filt = df[['zipcode', 'time', 'value']]
df_filt = df_filt.loc[(df_filt['time'].dt.year == 2019) | (df_filt['time'].dt.year == 2016)]
df_filt = df_filt.loc[(df_filt['time'].dt.month == 12)]
df_filt['time'].value_counts()

2019-12-31    30443
2016-12-31    29902
Name: time, dtype: int64

In [55]:
#Create smaller dfs, one for 2016 and one for 2019.
df16 = df_filt.loc[(df_filt['time'].dt.year == 2016)]
df16.info()

df19 = df_filt.loc[(df_filt['time'].dt.year == 2019)]
df19.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29902 entries, 7641193 to 7671635
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   zipcode  29902 non-null  int64         
 1   time     29902 non-null  datetime64[ns]
 2   value    29902 non-null  int64         
dtypes: datetime64[ns](1), int64(2)
memory usage: 934.4 KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 30443 entries, 8737141 to 8767583
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   zipcode  30443 non-null  int64         
 1   time     30443 non-null  datetime64[ns]
 2   value    30443 non-null  int64         
dtypes: datetime64[ns](1), int64(2)
memory usage: 951.3 KB


In [56]:
#Merge together with value columns renamed after month
df16['val_2016'] = df16['value']
df16.drop(['value', 'time'], axis=1, inplace=True)

df19['val_2019'] = df19['value']
df19.drop(['value', 'time'], axis=1, inplace=True)

growth_df = df16.merge(df19, how='inner', on='zipcode')

/Users/jamesfay/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/jamesfay/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [58]:
growth_df['zipcode'].nunique()

29902

In [60]:
growth_df['percent_growth'] = (growth_df['val_2019'] - growth_df['val_2016']) / growth_df['val_2016'] * 100
growth_df.sort_values(by='percent_growth', ascending=False)

,zipcode,val_2016,val_2019,percent_growth
7955,43205,85176,203412,138.813750
6422,30314,73819,172508,133.690513
3255,30310,115394,240692,108.582769
1021,30032,100903,199980,98.190341
2129,19132,24423,48190,97.314007
...,...,...,...,...
16204,70083,170765,123765,-27.523204
29690,31039,158507,114663,-27.660608
27095,1094,406925,291216,-28.434970
13493,39175,101576,71419,-29.689100


# Step 6: Interpreting Results